## Import libs

In [1]:
import requests as req
import numpy as np
import pandas as pd
import json

import warnings
warnings.filterwarnings('ignore')

## Connecting to isbndb API 

In [2]:
#home page: https://isbndb.com/
#doc: https://isbndb.com/apidocs/v2

h = {'Authorization': '48848_0d5c41033956592917fb92bcd923b67a'}

## Import Kaggle data books

In [84]:
dfBooks = pd.read_csv('Books.csv',low_memory=False)
dfUsers = pd.read_csv('Ratings.csv',low_memory=False)
dfRating = pd.read_csv('Users.csv',low_memory=False)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...


In [116]:
dfBooks2 = pd.read_excel('Books2.xlsx')
dfBooks2.shape

(2499, 19)

## Collecting data

In [117]:
#Slicing ISBN
dfList = dfBooks.iloc[0:, 0:1]
dfList2 = dfBooks2.iloc[0:, 12:14]
dfList2.rename(columns = {'isbn':'ISBN'},inplace = True)

dfList3 = dfList.merge(dfList2, how = 'left', on = 'ISBN')
dfList4 = dfList3[dfList3['isbn10'].isnull()]

In [118]:
#list size of search
group_size = 500

#Slicing ISBN
dfList4 = dfList4.iloc[0:group_size, 0:1]

#List of ISBN
dfList4 = dfList4['ISBN'].values.tolist()

#batch search
df = pd.DataFrame()
for i in dfList4:
    resp = req.get(f"https://api2.isbndb.com/book/{i}", headers=h).json()
    x = pd.DataFrame.from_dict(resp, 'index')
    df = df.append(x,ignore_index=True)

In [119]:
df.drop(columns = 0,inplace = True)
df1 = pd.concat([dfBooks2,df])
df1.shape

(2999, 19)

In [120]:
#Export
df1.to_excel("Books2.xlsx",index = False)
df1.head(1)

,publisher,language,image,title_long,edition,dimensions,date_published,authors,title,isbn13,msrp,binding,isbn,isbn10,pages,synopsis,subjects,related,other_isbns
0,Harper Uk,en,https://images.isbndb.com/covers/79/76/9780001...,Brave New World,Unabridged,"Height: 5.5 Inches, Length: 4.75 Inches, Weigh...",1999.0,"['Huxley, Aldous']",Brave New World,9780001047976,0.0,Audio Cassette,0001047973,0001047973,NaN,NaN,NaN,NaN,NaN
